In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
#!conda install -y psycopg2

In [ ]:
import dask
import dask.bag as db
import dask.dataframe as dd
import os
import pathlib
import pandas as pd
import xml.dom.minidom
import seaborn as sns

import psycopg2
import psycopg2.extras

In [ ]:
DATA_DIR = pathlib.Path(os.getenv('DATA_DIR'))
SWOB_DIR = DATA_DIR / 'data/test_download/swob/'

In [ ]:
def string_to_dict(obs_xml_string):
    obs_data = xml.dom.minidom.parseString(obs_xml_string)
    metadata = obs_data.getElementsByTagName('identification-elements')[0]

    metadata_dict = {}

    for element in metadata.childNodes:
        variable = element.attributes['name'].value
        value = element.attributes['value'].value
        metadata_dict[variable] = value

    obs_dict = {}

    elements = obs_data.getElementsByTagName('elements')[0]
    for element in elements.childNodes:
        variable = element.attributes['name'].value
        value = element.attributes['value'].value
        obs_dict[variable] = value

    return {**metadata_dict, **obs_dict}

In [ ]:
bag = db.read_text(DATA_DIR / 'data/test_download/swob/20201129/*/*.xml')

In [ ]:
obs_dicts = bag.map(string_to_dict)

In [ ]:
connection = psycopg2.connect(database='obs', user='postgres', password=None, host='localhost', port=5432)


In [ ]:
cursor = connection.cursor()
psycopg2.extras.execute_values(
    cursor, 
    """INSERT 
    INTO obs(network, valid_time, latitude, longitude, elevation, air_temperature, station_pressure, visibility, wmo_synop_id, geog) 
    VALUES %s""",
    obs_dicts, template="('swob', %(date_tm)s, %(lat)s, %(long)s, %(stn_elev)s, %(air_temp)s, %(stn_pres)s, %(vis)s, %(wmo_synop_id)s, ST_SetSRID(ST_MakePoint(%(long)s, %(lat)s),4326))")

In [ ]:
connection.commit()
